<h1 align = 'center'> Heart Attack Analysis And Prediction </h1>

## Project Content
1. [Introdction](#1)

    *1.1 [topic](#2)

## Introduction <a id=1></a>

### Variable Description
- Age: age in years<br />
- Sex: Sex of the patient<br />
    - Value 1: male<br />
    - Value 0: female<br />
- exang: exercise induced angina<br />
    - Value 1: yes<br />
    - Value 0: no<br />
- ca: number of major vessels (0-3)<br />
- cp: Chest Pain type chest pain type<br />
    - Value 1: typical angina<br />
    - Value 2: atypical angina<br />
    - Value 3: non-anginal pain<br />
    - Value 4: asymptomatic<br />
- trtbps: resting blood pressure (in mm Hg)<br />
- chol: cholestoral in mg/dl fetched via BMI sensor<br />
- fbs: (fasting blood sugar > 120 mg/dl)<br />
    - Value 1: true<br />
    - Value 0: false<br />
- restecg: resting electrocardiographic results<br />
    - Value 0: normal<br />
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)<br />
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria<br />
- thalach: maximum heart rate achieved<br />
- num: the predicted attribute - diagnosis of heart disease (angiographic disease status)<br />
    - Value 1: > 50% diameter narrowing<br />
    - Value 0: < diameter narrowing<br />
- slope: the slope of the peak exercise ST segment<br />
    - Value 1: flat<br />
    - Value 2: upsloping<br />
    - Value 0: downsloping<br />
- target: <br />
    - Value 1: more chance of heart attack<br />
    - Value 0: less chance of heart attack<br />

#### Required Libraries

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [6]:
url = "https://raw.githubusercontent.com/audrey-rw5683/HeartAttackPrediction/main/heart.csv"
df = pd.read_csv(url)
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
